We need to make the cooccurrence matrices to then calculate the pointwise mutual information (PMI) values to pass to the Dynamic Word Embedding model. 

In [1]:
from csv import DictReader
from collections import Counter

# read in the freq counter
totalFreqDistFile = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/totalFreqDist.csv'

with open(totalFreqDistFile) as f:
    reader = DictReader(f, fieldnames=['word', 'count'])
    freqDist = Counter({row['word']: int(row['count']) for row in reader})

print(freqDist.most_common(10))
            

[('god', 3156851), ('one', 2268075), ('upon', 2256671), ('may', 2243437), ('shall', 2129379), ('us', 1831379), ('man', 1797155), ('would', 1713276), ('great', 1654792), ('men', 1574743)]


In [2]:
print(sum(freqDist.values()))
# 463,231,888 words

463231888


In [3]:
frequentWords = [w for w in freqDist.keys() if freqDist[w] > 200]
# 71670, 3x than paper

In [4]:
print(len(frequentWords))
print(frequentWords[0:10])

71670
['god', 'one', 'upon', 'may', 'shall', 'us', 'man', 'would', 'great', 'men']


In [5]:
# create and save hashmap of word to ids
wordToID = dict(zip(frequentWords, range(len(freqDist))))

In [6]:
IDToWord = dict([(wordToID[word], word) for word in wordToID.keys()])

In [7]:
import os
from os import listdir
from os.path import isfile, join

textPath = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/corpus/cleaned_txt/'

textFiles = [f for f in listdir(textPath) if isfile(join(textPath, f))]

len(textFiles)

32830

In [8]:
import numpy as np
from scipy.sparse import dok_matrix
df = dok_matrix((len(frequentWords), len(frequentWords)))

In [9]:
df.shape
# df['god']
# df['god']['god'] should store the total number of appearances
# df will be a diagonal matrix

(71670, 71670)

In [11]:
def asdf( args ):

    word = args[0] 
    window = args[1] 
    wordMap = args[3]
    
    for coword in window:
        try:
            wordID = wordMap[word]
            cowordID = wordMap[coword]
            (args[2])[wordID, cowordID] += 1.0
        except:
            print(word)

In [12]:
import nltk

cooccurrencePath = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/cooccur/'
L = 5 # window size, or L in the paper
# window: x1 x2 ... xL W y1 y2 ... yL

# https://stackoverflow.com/questions/17458751/python-symmetric-word-matrix-using-nltk
# need to parallelize

with open(textPath + textFiles[0], 'r+') as f:
    text = f.read()
    tokens = [w for w in nltk.word_tokenize(text) if w in frequentWords]
#     for i in range(L, len(tokens)-1):
    for i in range(L, L+2):
        word = tokens[i]
        window = tokens[i-L : i + L + 1]
        asdf((word, window, df, wordToID))
          

In [14]:
print(df)

  (17048, 1779)	1.0
  (17048, 2746)	1.0
  (17048, 2314)	1.0
  (17048, 891)	1.0
  (17048, 3265)	1.0
  (17048, 17048)	1.0
  (17048, 2099)	1.0
  (17048, 682)	1.0
  (17048, 600)	1.0
  (17048, 1066)	1.0
  (17048, 39257)	1.0
  (2099, 2746)	1.0
  (2099, 2314)	1.0
  (2099, 891)	1.0
  (2099, 3265)	1.0
  (2099, 17048)	1.0
  (2099, 2099)	1.0
  (2099, 682)	1.0
  (2099, 600)	1.0
  (2099, 1066)	1.0
  (2099, 39257)	1.0
  (2099, 115)	1.0
